# Transfer Learning mit FARM

## Ziele


*   BERT und Transfer Learning
*   Einführung in FARM

*   Wie funktioniert das Fine-Tuning eines BERT-Modells?





## BERT und Transfer Learning
Transfer Learning ist eine Machine Learning-Methode, bei der ein Modell, das für einen bestimmten Task trainiert wurde, als Ausgangspunkt für ein Modell für einen anderen Task verwendet wird.

Die Idee hinter diesem Konzept ist, dass man das Erlernte von einem Problem auf ein anderes überträgt. Wenn man z.B. Java als erste Programmiersprache gelernt hat, fällt es uns leichter, Python oder eine andere Programmiersprache zu erlernen. Denn die grundlegenden Programmierkonzepte aus Java/OOP können leicht auf jede andere Programmiersprache übertragen werden. Man muss also nicht von Null beginnnen, sondern kann auf existierendem Wissen aufbauen. Ähnlich verhält es sich auch mit dem Transfer Learning von Sprachmodellen wie BERT. Existierende BERT-Modelle wurden hauptsächlich auf der Wikipedia trainiert. Möchte man allerdings Textklassifikationen in bestimmten Domänen durchführen, ist die Wikipedia mit ihrem Vokabular nicht repräsentativ genug. Daher nutzt man das Fine-Tuning von vortrainierten BERT-Modellen, um sie an den neuen Task/das neue Problem anzupassen. Durch diesen Schritt erhofft man sich bessere Performance.

## FARM 
(**F**ramework for **A**dapting **R**epresentation **M**odels)
![farm_logo](https://github.com/deepset-ai/FARM/raw/master/docs/img/farm_logo_text_right_wide.png?raw=true)
... ermöglicht uns, das Transfer Learning mit BERT (und anderen Sprachmodellen) umzusetzen. Damit kann man einfach und schnell Modelle für Tasks wie Text Classification, NER und Question Answering erstellen:
![task_table](https://miro.medium.com/max/3840/1*0RYwLSOMegTKfhnyTwkjIQ.png)

# Fine-Tuning eines BERT-Modells mit FARM (adapted from FARM-Tutorial 1)

Welcome to the FARM building blocks tutorial! There are many different ways to make use of this repository, but in this notebook, we will be going through the most import building blocks that will help you harvest the rewards of a successfully trained NLP model.

Happy FARMing!

## 1) Text Classification

GermEval 2018 (GermEval2018) (https://projects.fzai.h-da.de/iggsa/) is an open data set containing texts that need to be classified by whether they are offensive or not. There are a set of coarse and fine labels, but here we will only be looking at the coarse set which labels each example as either OFFENSE or OTHER. To tackle this task, we are going to build a classifier that is composed of Google's BERT language model and a feed forward neural network prediction head.

### Setup

In [1]:
# Install FARM
!pip install torch==1.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install farm==0.5.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
# Here are the imports we need

import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger


09/13/2021 16:00:53 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [3]:
# Farm allows simple logging of many parameters & metrics. Let's use the MLflow framework to track our experiment ...
# You will see your results on https://public-mlflow.deepset.ai/
ml_logger = MLFlowLogger(tracking_uri="https://public-mlflow.deepset.ai/")
ml_logger.init_experiment(experiment_name="Public_FARM", run_name="Tutorial1_Colab")



 __          __  _                            _        
 \ \        / / | |                          | |       
  \ \  /\  / /__| | ___ ___  _ __ ___   ___  | |_ ___  
   \ \/  \/ / _ \ |/ __/ _ \| '_ ` _ \ / _ \ | __/ _ \ 
    \  /\  /  __/ | (_| (_) | | | | | |  __/ | || (_) |
     \/  \/ \___|_|\___\___/|_| |_| |_|\___|  \__\___/ 
  ______      _____  __  __  
 |  ____/\   |  __ \|  \/  |              _.-^-._    .--.
 | |__ /  \  | |__) | \  / |           .-'   _   '-. |__|
 |  __/ /\ \ |  _  /| |\/| |          /     |_|     \|  |
 | | / ____ \| | \ \| |  | |         /               \  |
 |_|/_/    \_\_|  \_\_|  |_|        /|     _____     |\ |
                                     |    |==|==|    |  |
|---||---|---|---|---|---|---|---|---|    |--|--|    |  |
|---||---|---|---|---|---|---|---|---|    |==|==|    |  |
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 


In [4]:
# We need to fetch the right device to drive the growth of our model
# Make sure that you have gpu turned on in this notebook by going to
# Runtime>Change runtime type and select GPU as Hardware accelerator.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Devices available: {}".format(device))

Devices available: cuda


### Data Handling

In FARM the Processor contains the functions which handle the conversion from file or request to PyTorch Datasets. In essence, it prepares data to be consumed by the modelling components. This is done in stages to allow for easier debugging. It should be able to handle file input or requests. This class contains everything that needs to be customized when adapting a new dataset. Custom datasets can be handled by extending the Processor (e.g. see CONLLProcessor).

The DataSilo is a generic class that stores the train, dev and test data sets. It calls upon the methods from the Processor to do the loading and then exposes a DataLoader for each set. In cases where there is not a separate dev file, it will create one by slicing the train set.
![data_handling](https://farm.deepset.ai/_images/data_silo_no_bg.jpg)

## distilbert-base-uncased-finetuned-sst-2-english

In [5]:
# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

tokenizer = Tokenizer.load(
    pretrained_model_name_or_path="distilbert-base-uncased-finetuned-sst-2-english",
    do_lower_case=False)




09/13/2021 16:00:54 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'DistilBertTokenizer'


In [6]:
# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor
# TOXIC = 1
# OTHER = 0
LABEL_LIST = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        warmup = 600,
                                        train_filename="train.tsv",
                                        test_filename="test_with_solutions.tsv",
                                        data_dir="../content",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        text_column_name="Comment",
                                        label_column_name="Insult")

In [7]:
# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

09/13/2021 16:00:56 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
09/13/2021 16:00:56 - INFO - farm.data_handler.data_silo -   Loading train set from: ../content/train.tsv 
09/13/2021 16:00:57 - INFO - farm.data_handler.data_silo -   Got ya 1 parallel workers to convert 3947 dictionaries to pytorch datasets (chunksize = 790)...
09/13/2021 16:00:57 - INFO - farm.data_handler.data_silo -    0 
09/13/2021 16:00:57 - INFO - farm.data_handler.data_silo -   /|\
09/13/2021 16:00:57 - INFO - farm.data_handler.data_silo -   /'\
09/13/2021 16:00:57 - INFO - farm.data_handler.data_silo -   
Preprocessing Dataset ../content/train.tsv:   0%|          | 0/3947 [00:00<?, ? Dicts/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils.py:460: FutureWarning: `is_pretokenized` is deprecated and will be removed in 

### Modeling

![modeling](https://farm.deepset.ai/_images/adaptive_model_no_bg.jpg)
In FARM, we make a strong distinction between the language model and prediction head so that you can mix and match different building blocks for your needs.

For example, in the transfer learning paradigm, you might have the one language model that you will be using for both document classification and NER. Or perhaps you have a pretrained language model which you would like to adapt to your domain, then use it for a downstream task such as question answering. 

All this is possible within FARM and requires only the replacement of a few modular components, as we shall see below.

Let's first have a look at how we might set up a model.

In [8]:
# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "distilbert-base-uncased-finetuned-sst-2-english"
# MODEL_NAME_OR_PATH = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

09/13/2021 16:01:17 - INFO - farm.modeling.language_model -   Automatically detected language from language model name: english


In [9]:
# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

09/13/2021 16:01:17 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
09/13/2021 16:01:17 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [0.6820535 1.8732227]


In [10]:
# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
# EMBEDS_DROPOUT_PROB = 0.1 distilbert-base-uncased-finetuned-sst-2-english_1.PNG
# EMBEDS_DROPOUT_PROB = 0.01
# EMBEDS_DROPOUT_PROB = 0.2 distilbert-base-uncased-finetuned-sst-2-english_2.PNG
EMBEDS_DROPOUT_PROB = 0.01

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

### Training

In [11]:
# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

#LEARNING_RATE = 2e-5 # distilbert-base-uncased-finetuned-sst-2-english_1.PNG
#LEARNING_RATE = 1e-7
LEARNING_RATE = 1e-5  # distilbert-base-uncased-finetuned-sst-2-english_2.PNG

N_EPOCHS = 3

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

09/13/2021 16:01:19 - INFO - farm.modeling.optimization -   Loading optimizer `TransformersAdamW`: '{'correct_bias': False, 'weight_decay': 0.01, 'lr': 1e-05}'
09/13/2021 16:01:19 - INFO - farm.modeling.optimization -   Using scheduler 'get_linear_schedule_with_warmup'
09/13/2021 16:01:19 - INFO - farm.modeling.optimization -   Loading schedule `get_linear_schedule_with_warmup`: '{'num_warmup_steps': 29.700000000000003, 'num_training_steps': 297}'


In [12]:
# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device, 
)

In [13]:
!nvidia-smi

Mon Sep 13 16:01:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    60W / 149W |    628MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:

model = trainer.train()

09/13/2021 16:01:20 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 25/25 [00:05<00:00,  4.57it/s]
09/13/2021 16:02:30 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EV

In [15]:
# Test your model on a sample (Inference)
from farm.infer import Inferencer
from pprint import PrettyPrinter

infer_model = Inferencer(processor=processor, model=model, task_type="text_classification", gpu=True)

basic_texts = [
    {"text": "Martin is an idiot"},
    {"text": "Martin Müller plays Voleyball in Berlin"},
]
result = infer_model.inference_from_dicts(dicts=basic_texts)
PrettyPrinter().pprint(result)


09/13/2021 16:04:57 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
09/13/2021 16:04:57 - INFO - farm.infer -   Got ya 1 parallel workers to do inference ...
09/13/2021 16:04:57 - INFO - farm.infer -    0 
09/13/2021 16:04:57 - INFO - farm.infer -   /w\
09/13/2021 16:04:57 - INFO - farm.infer -   /'\
09/13/2021 16:04:57 - INFO - farm.infer -   
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils.py:460: FutureWarning: `is_pretokenized` is deprecated and will be removed in a future version, use `is_split_into_words` instead.
  FutureWarning,
09/13/2021 16:04:57 - INFO - farm.data_handler.processor -   *** Show 2 random examples ***
09/13/2021 16:04:57 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ __ ___  _ __ | | ___ 
      "||"       \___ \ / _` | '_ ` _ \| '_ \| |

[{'predictions': [{'context': 'Martin is an idiot',
                   'end': None,
                   'label': '1',
                   'probability': 0.9937976,
                   'start': None},
                  {'context': 'Martin Müller plays Voleyball in Berlin',
                   'end': None,
                   'label': '0',
                   'probability': 0.89760894,
                   'start': None}],
  'task': 'text_classification'}]


In [16]:
#model.save('../data/detecting-insults-in-social-commentary/model')
#processor.save('../data/detecting-insults-in-social-commentary/processor')

## Aufgabe
Verwenden Sie den imdb-Datensatz aus der letzten Sitzung und nutzen Sie ein englisches BERT-Modell (z.B. ```bert-base-uncased```), um es für die Sentiment-Analyse von Film-Reviews finezutunen. Bilden Sie Gruppen und gehen Sie wie folgt vor:

1. Verwenden Sie ein Subset des Datensatzes. Grund: Es ist unklar, wie viel die (kostenlose) Google Colab-GPU zu leisten im Stande ist. Verwenden Sie innerhalb der Gruppe unterschiedliche Samplezahlen des Datensatzes und testen Sie aus, wo die Grenzen liegen. 
2. Adjustieren Sie verschiedene Parameter (Learning Rate, Dropout Rate ...), um Unterschiede in der Performance festzustellen.
3. Speichern Sie das fine-getunte BERT-Modell ab: 
```python
model.save('path/to/directory')
processor.save('path/to/directory')
```


## BERT <br>
https://huggingface.co/bert-base-uncased 

In [17]:
from transformers import AutoTokenizer
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

#tokenizer = Tokenizer.load(
#    pretrained_model_name_or_path="finiteautomata/bertweet-base-sentiment-analysis",
#    do_lower_case=False)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor
# TOXIC = 1
# OTHER = 0
LABEL_LIST = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        warmup = 600,
                                        train_filename="train.tsv",
                                        test_filename="test_with_solutions.tsv",
                                        data_dir="../content",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        text_column_name="Comment",
                                        label_column_name="Insult")

# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "bert-base-uncased"
# MODEL_NAME_OR_PATH = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
# EMBEDS_DROPOUT_PROB = 0.1 distilbert-base-uncased-finetuned-sst-2-english_1.PNG
# EMBEDS_DROPOUT_PROB = 0.01
# EMBEDS_DROPOUT_PROB = 0.2 distilbert-base-uncased-finetuned-sst-2-english_2.PNG
EMBEDS_DROPOUT_PROB = 0.01

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

#LEARNING_RATE = 2e-5 # distilbert-base-uncased-finetuned-sst-2-english_1.PNG
#LEARNING_RATE = 1e-7
LEARNING_RATE = 1e-5  # distilbert-base-uncased-finetuned-sst-2-english_2.PNG

N_EPOCHS = 3

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device, 
)

!nvidia-smi

model = trainer.train()

09/13/2021 16:04:59 - WARNING - farm.utils -   Failed to log params: INVALID_PARAMETER_VALUE: Changing param value is not allowed. Param with key='tokenizer' was already logged with value='DistilBertTokenizer' for run ID='6543ccfdbba146ca82a34b6de1bbb983. Attempted logging new value 'BertTokenizer'.
09/13/2021 16:04:59 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
09/13/2021 16:04:59 - INFO - farm.data_handler.data_silo -   Loading train set from: ../content/train.tsv 
09/13/2021 16:04:59 - INFO - farm.data_handler.data_silo -   Got ya 1 parallel workers to convert 3947 dictionaries to pytorch datasets (chunksize = 790)...
09/13/2021 16:04:59 - INFO - farm.data_handler.data_silo -    0 
09/13/2021 16:04:59 - INFO - farm.data_handler.data_silo -   /w\
09/13/2021 16:04:59 - INFO - farm.data_handler.data_silo -   / 

Mon Sep 13 16:05:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    60W / 149W |   3237MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

09/13/2021 16:05:28 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 25/25 [00:10<00:00,  2.28it/s]
09/13/2021 16:07:45 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EV

## RoBERTa <br>
https://huggingface.co/roberta-base 

In [18]:
from transformers import AutoTokenizer
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

#tokenizer = Tokenizer.load(
#    pretrained_model_name_or_path="finiteautomata/bertweet-base-sentiment-analysis",
#    do_lower_case=False)

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor
# TOXIC = 1
# OTHER = 0
LABEL_LIST = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        warmup = 600,
                                        train_filename="train.tsv",
                                        test_filename="test_with_solutions.tsv",
                                        data_dir="../content",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        text_column_name="Comment",
                                        label_column_name="Insult")

# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "roberta-base"
# MODEL_NAME_OR_PATH = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
# EMBEDS_DROPOUT_PROB = 0.1 distilbert-base-uncased-finetuned-sst-2-english_1.PNG
# EMBEDS_DROPOUT_PROB = 0.01
# EMBEDS_DROPOUT_PROB = 0.2 distilbert-base-uncased-finetuned-sst-2-english_2.PNG
EMBEDS_DROPOUT_PROB = 0.01

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

#LEARNING_RATE = 2e-5 # distilbert-base-uncased-finetuned-sst-2-english_1.PNG
#LEARNING_RATE = 1e-7
LEARNING_RATE = 1e-5  # distilbert-base-uncased-finetuned-sst-2-english_2.PNG

N_EPOCHS = 3

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device, 
)

!nvidia-smi

model = trainer.train()

09/13/2021 16:12:37 - WARNING - farm.utils -   Failed to log params: INVALID_PARAMETER_VALUE: Changing param value is not allowed. Param with key='tokenizer' was already logged with value='DistilBertTokenizer' for run ID='6543ccfdbba146ca82a34b6de1bbb983. Attempted logging new value 'RobertaTokenizer'.
09/13/2021 16:12:37 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
09/13/2021 16:12:37 - INFO - farm.data_handler.data_silo -   Loading train set from: ../content/train.tsv 
09/13/2021 16:12:37 - INFO - farm.data_handler.data_silo -   Got ya 1 parallel workers to convert 3947 dictionaries to pytorch datasets (chunksize = 790)...
09/13/2021 16:12:37 - INFO - farm.data_handler.data_silo -    0 
09/13/2021 16:12:37 - INFO - farm.data_handler.data_silo -   /w\
09/13/2021 16:12:37 - INFO - farm.data_handler.data_silo -  

Mon Sep 13 16:13:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    60W / 149W |   6289MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

09/13/2021 16:13:10 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 25/25 [00:10<00:00,  2.29it/s]
09/13/2021 16:15:28 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EV

## ALBERT <br>
https://huggingface.co/albert-base-v2

In [19]:
from transformers import AutoTokenizer
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

#tokenizer = Tokenizer.load(
#    pretrained_model_name_or_path="finiteautomata/bertweet-base-sentiment-analysis",
#    do_lower_case=False)

tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor
# TOXIC = 1
# OTHER = 0
LABEL_LIST = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        warmup = 600,
                                        train_filename="train.tsv",
                                        test_filename="test_with_solutions.tsv",
                                        data_dir="../content",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        text_column_name="Comment",
                                        label_column_name="Insult")

# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "albert-base-v2"
# MODEL_NAME_OR_PATH = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
# EMBEDS_DROPOUT_PROB = 0.1 distilbert-base-uncased-finetuned-sst-2-english_1.PNG
# EMBEDS_DROPOUT_PROB = 0.01
# EMBEDS_DROPOUT_PROB = 0.2 distilbert-base-uncased-finetuned-sst-2-english_2.PNG
EMBEDS_DROPOUT_PROB = 0.01

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

#LEARNING_RATE = 2e-5 # distilbert-base-uncased-finetuned-sst-2-english_1.PNG
#LEARNING_RATE = 1e-7
LEARNING_RATE = 1e-5  # distilbert-base-uncased-finetuned-sst-2-english_2.PNG

N_EPOCHS = 3

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device, 
)

!nvidia-smi

model = trainer.train()

09/13/2021 16:20:20 - WARNING - farm.utils -   Failed to log params: INVALID_PARAMETER_VALUE: Changing param value is not allowed. Param with key='tokenizer' was already logged with value='DistilBertTokenizer' for run ID='6543ccfdbba146ca82a34b6de1bbb983. Attempted logging new value 'AlbertTokenizer'.
09/13/2021 16:20:20 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
09/13/2021 16:20:20 - INFO - farm.data_handler.data_silo -   Loading train set from: ../content/train.tsv 
09/13/2021 16:20:20 - INFO - farm.data_handler.data_silo -   Got ya 1 parallel workers to convert 3947 dictionaries to pytorch datasets (chunksize = 790)...
09/13/2021 16:20:20 - INFO - farm.data_handler.data_silo -    0 
09/13/2021 16:20:20 - INFO - farm.data_handler.data_silo -   /w\
09/13/2021 16:20:20 - INFO - farm.data_handler.data_silo -   

Mon Sep 13 16:20:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    62W / 149W |   6881MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

09/13/2021 16:20:37 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 25/25 [00:12<00:00,  2.08it/s]
09/13/2021 16:23:00 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EV

## DistilBERT <br>
https://huggingface.co/distilbert-base-uncased

In [20]:
from transformers import AutoTokenizer
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

#tokenizer = Tokenizer.load(
#    pretrained_model_name_or_path="finiteautomata/bertweet-base-sentiment-analysis",
#    do_lower_case=False)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor
# TOXIC = 1
# OTHER = 0
LABEL_LIST = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        warmup = 600,
                                        train_filename="train.tsv",
                                        test_filename="test_with_solutions.tsv",
                                        data_dir="../content",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        text_column_name="Comment",
                                        label_column_name="Insult")

# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "distilbert-base-uncased"
# MODEL_NAME_OR_PATH = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
# EMBEDS_DROPOUT_PROB = 0.1 distilbert-base-uncased-finetuned-sst-2-english_1.PNG
# EMBEDS_DROPOUT_PROB = 0.01
# EMBEDS_DROPOUT_PROB = 0.2 distilbert-base-uncased-finetuned-sst-2-english_2.PNG
EMBEDS_DROPOUT_PROB = 0.01

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

#LEARNING_RATE = 2e-5 # distilbert-base-uncased-finetuned-sst-2-english_1.PNG
#LEARNING_RATE = 1e-7
LEARNING_RATE = 1e-5  # distilbert-base-uncased-finetuned-sst-2-english_2.PNG

N_EPOCHS = 3

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device, 
)

!nvidia-smi

model = trainer.train()

09/13/2021 16:28:08 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
09/13/2021 16:28:08 - INFO - farm.data_handler.data_silo -   Loading train set from: ../content/train.tsv 
09/13/2021 16:28:08 - INFO - farm.data_handler.data_silo -   Got ya 1 parallel workers to convert 3947 dictionaries to pytorch datasets (chunksize = 790)...
09/13/2021 16:28:08 - INFO - farm.data_handler.data_silo -    0 
09/13/2021 16:28:08 - INFO - farm.data_handler.data_silo -   /|\
09/13/2021 16:28:08 - INFO - farm.data_handler.data_silo -   /'\
09/13/2021 16:28:08 - INFO - farm.data_handler.data_silo -   
Preprocessing Dataset ../content/train.tsv:   0%|          | 0/3947 [00:00<?, ? Dicts/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils.py:460: FutureWarning: `is_pretokenized` is deprecated and will be removed in 

Mon Sep 13 16:28:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    60W / 149W |   6881MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

09/13/2021 16:28:36 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 25/25 [00:05<00:00,  4.58it/s]
09/13/2021 16:29:46 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EV

## XLMRoBERTa <br>
https://huggingface.co/xlm-roberta-base 

In [21]:
from transformers import AutoTokenizer
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

#tokenizer = Tokenizer.load(
#    pretrained_model_name_or_path="finiteautomata/bertweet-base-sentiment-analysis",
#    do_lower_case=False)

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor
# TOXIC = 1
# OTHER = 0
LABEL_LIST = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        warmup = 600,
                                        train_filename="train.tsv",
                                        test_filename="test_with_solutions.tsv",
                                        data_dir="../content",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        text_column_name="Comment",
                                        label_column_name="Insult")

# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "xlm-roberta-base"
# MODEL_NAME_OR_PATH = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
# EMBEDS_DROPOUT_PROB = 0.1 distilbert-base-uncased-finetuned-sst-2-english_1.PNG
# EMBEDS_DROPOUT_PROB = 0.01
# EMBEDS_DROPOUT_PROB = 0.2 distilbert-base-uncased-finetuned-sst-2-english_2.PNG
EMBEDS_DROPOUT_PROB = 0.01

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

#LEARNING_RATE = 2e-5 # distilbert-base-uncased-finetuned-sst-2-english_1.PNG
#LEARNING_RATE = 1e-7
LEARNING_RATE = 1e-5  # distilbert-base-uncased-finetuned-sst-2-english_2.PNG

N_EPOCHS = 3

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device, 
)

!nvidia-smi

model = trainer.train()

09/13/2021 16:32:15 - WARNING - farm.utils -   Failed to log params: INVALID_PARAMETER_VALUE: Changing param value is not allowed. Param with key='tokenizer' was already logged with value='DistilBertTokenizer' for run ID='6543ccfdbba146ca82a34b6de1bbb983. Attempted logging new value 'XLMRobertaTokenizer'.
09/13/2021 16:32:15 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
09/13/2021 16:32:15 - INFO - farm.data_handler.data_silo -   Loading train set from: ../content/train.tsv 
09/13/2021 16:32:15 - INFO - farm.data_handler.data_silo -   Got ya 1 parallel workers to convert 3947 dictionaries to pytorch datasets (chunksize = 790)...
09/13/2021 16:32:15 - INFO - farm.data_handler.data_silo -    0 
09/13/2021 16:32:15 - INFO - farm.data_handler.data_silo -   /w\
09/13/2021 16:32:15 - INFO - farm.data_handler.data_silo 

Mon Sep 13 16:32:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    60W / 149W |   7615MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

09/13/2021 16:32:44 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 25/25 [00:10<00:00,  2.29it/s]
09/13/2021 16:35:13 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EV

## XLNet Cased <br>
https://huggingface.co/xlnet-base-cased 

In [ ]:
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

#tokenizer = Tokenizer.load(
#    pretrained_model_name_or_path="finiteautomata/bertweet-base-sentiment-analysis",
#    do_lower_case=False)

tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor
# TOXIC = 1
# OTHER = 0
LABEL_LIST = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        warmup = 600,
                                        train_filename="train.tsv",
                                        test_filename="test_with_solutions.tsv",
                                        data_dir="../content",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        text_column_name="Comment",
                                        label_column_name="Insult")

# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "xlnet-base-cased"
# MODEL_NAME_OR_PATH = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
# EMBEDS_DROPOUT_PROB = 0.1 distilbert-base-uncased-finetuned-sst-2-english_1.PNG
# EMBEDS_DROPOUT_PROB = 0.01
# EMBEDS_DROPOUT_PROB = 0.2 distilbert-base-uncased-finetuned-sst-2-english_2.PNG
EMBEDS_DROPOUT_PROB = 0.01

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

#LEARNING_RATE = 2e-5 # distilbert-base-uncased-finetuned-sst-2-english_1.PNG
#LEARNING_RATE = 1e-7
LEARNING_RATE = 1e-5  # distilbert-base-uncased-finetuned-sst-2-english_2.PNG

N_EPOCHS = 3

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device, 
)

!nvidia-smi

model = trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/configuration_xlnet.py:212: FutureWarning: This config doesn't use attention memories, a core feature of XLNet. Consider setting `men_len` to a non-zero value, for example `xlnet = XLNetLMHeadModel.from_pretrained('xlnet-base-cased'', mem_len=1024)`, for accurate training performance as well as an order of magnitude faster inference. Starting from version 3.5.0, the default parameter will be 1024, following the implementation in https://arxiv.org/abs/1906.08237
  FutureWarning,
09/13/2021 16:40:25 - WARNING - farm.utils -   Failed to log params: INVALID_PARAMETER_VALUE: Changing param value is not allowed. Param with key='tokenizer' was already logged with value='DistilBertTokenizer' for run ID='6543ccfdbba146ca82a34b6de1bbb983. Attempted logging new value 'XLNetTokenizer'.
09/13/2021 16:40:25 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`

## Switching to NER (NOT WORKING)

In [ ]:
# Import the new building blocks

from farm.data_handler.processor import NERProcessor
from farm.modeling.prediction_head import TokenClassificationHead
ml_logger.init_experiment(experiment_name="Public_FARM", run_name="Tutorial1_Colab_NER")

# This processor will preprocess the data for the CoNLL03 NER task
ner_labels = ["[PAD]", "X", "O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-OTH", "I-OTH"]

ner_processor = NERProcessor(tokenizer=tokenizer, 
                             max_seq_len=128, 
                             data_dir="../data/conll03-de",
                             label_list=ner_labels,
                             metric="seq_f1")

# This prediction head is also a feed forward neural network but expects one
# vector per token in the input sequence and will generate a set of logits
# for each input

ner_prediction_head = TokenClassificationHead(num_labels=len(ner_labels))

# We can integrate these new pieces with the rest using this code
# It is pretty much the same structure as what we had above for text classification

BATCH_SIZE = 32
EMBEDS_DROPOUT_PROB = 0.1
LEARNING_RATE = 2e-5
N_EPOCHS = 1
N_GPU = 1

data_silo = DataSilo(
    processor=ner_processor,
    batch_size=BATCH_SIZE)

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[ner_prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_token"],
    device=device)

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS,
    device=device)

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device,
)

model = trainer.train()

In [ ]:
# Import the new building blocks

from farm.data_handler.processor import NERProcessor
from farm.modeling.prediction_head import TokenClassificationHead
ml_logger.init_experiment(experiment_name="Public_FARM", run_name="Tutorial1_Colab_NER")


In [ ]:
# This processor will preprocess the data for the CoNLL03 NER task
ner_labels = ["1", "0"]

ner_processor = NERProcessor(tokenizer=tokenizer, 
                             max_seq_len=128,
                             train_filename="train.tsv",
                             test_filename="test_with_solutions.tsv",
                             dev_filename=None,
                             data_dir="../content",
                             label_list=ner_labels,
                             metric="f1_macro",
                             text_column_name="Comment",
                              label_column_name="Insult")

In [ ]:
# This prediction head is also a feed forward neural network but expects one
# vector per token in the input sequence and will generate a set of logits
# for each input

ner_prediction_head = TokenClassificationHead(num_labels=len(ner_labels))

In [ ]:
# We can integrate these new pieces with the rest using this code
# It is pretty much the same structure as what we had above for text classification

BATCH_SIZE = 32
EMBEDS_DROPOUT_PROB = 0.1
LEARNING_RATE = 2e-5
N_EPOCHS = 1
N_GPU = 1

data_silo = DataSilo(
    processor=ner_processor,
    batch_size=BATCH_SIZE)

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[ner_prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS,
    device=device)

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device,
)

In [ ]:
model = trainer.train()